Семинар будет разделен на несколько частей. В части 1 мы разберем самостоятельную реализацию решающего дерева, а в части 2 - ознакомимся с соответствующим функционалом sklearn

## Часть 1. Решающее дерево с нуля

In [101]:
"""
    Простой датасет
    Задача - классифицировать фрукт по его цвету и диаметру.
    Колонка 1 - цвет
    Колонка 2 - диаметр фрукта (1 -
                маленький, 2 - средний, 3 - большой)
    Колонка 3 - название фрукта, целевая переменная
"""

header = ["color", "diameter", "label"]

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 2, 'Lemon'],
]

Также нам необходимо реализовать некоторые сопутствующие функции,
например:

- Кол-во уникальных значений в колонке
- Кол-во объектов каждого класса

In [13]:
def unique_values(rows, col):
    return {row[col] for row in rows}

def class_counts(rows):
    counts = {}
    
    for row in rows:
        label = row[2]
        if label in counts:
            counts[label] += 1
        else:
            counts[label] = 1
            
    return counts
            
def is_numeric(val):
    return isinstance(val, int) or isinstance(val, float)

In [10]:
unique_values(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [11]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [14]:
is_numeric(7)

True

In [15]:
is_numeric("Dummy string")

False

In [29]:
class Question:
    """
    Объект данного класса представляет из себя вопрос (или предикат,
    если выражаться математическим термином), с помощью которого
    происходит разбиение датасета.
    
    Записывает только номер колонки и ее значение. Метод match
    используется для сравнения значения признака с значением, по
    которому происходит сплит.

    Условимся, что если признак является численным, то происходит
    сравнение '>=', иначе - '=='
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        feature_value = example[self.column]
        
        if is_numeric(feature_value) != is_numeric(self.value):
            raise ValueException(f"Feature value and split value have different types - {type(self.value)}, {type(feature_value)}")
            
        if is_numeric(feature_value):
            return feature_value >= self.value
        else:
            return feature_value == self.value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        
        return f"Is {header[self.column]} {condition} {self.value}"

Теперь рассмотрим поведение Question в случае численного признака

In [30]:
Question(1, 3)

Is diameter >= 3

Аналогично для категориального

In [35]:
q = Question(2, "Apple")
q

Is label == Apple

Проверим как работает метод match

In [36]:
match_example = training_data[0]
q.match(match_example)

True

In [79]:
def split_partition(rows, split_question):
    """
    Разбить исходные записи rows при помощи
    вопроса (предиката) question на две части.
    Слева - те записи, для которых question.match == False,
    Справа - записи, для которых question.match == True
    """
    
    left_partition = []
    right_partition = []
    
    for row in rows:
        if split_question.match(row):
            right_partition.append(row)
        else:
            left_partition.append(row)
            
    return left_partition, right_partition

Сразу напишем небольшой тест нашего сплита

In [80]:
def test_split_partition():
    has_errors = False
    
    text_question = Question(0, "Red")
    false_rows, true_rows = split_partition(training_data, Question(0, "Red"))
    
    for row in false_rows:
        if row[0] == text_question.value:
            print("Error in false partition for text!")
            print(f"Left: {false_rows}")
            has_errors = True
    
    for row in true_rows:
        if row[0] != text_question.value:
            print("Error in true partition for text!")
            print(f"Right: {true_rows}")
            has_errors = True
            
    if has_errors:
        raise RuntimeError("Split partition test is not passed, stopping right now")
        
    """
    Тест для сплита численных колонок предлагаю написать самим
    """
        
test_split_partition()

**Что мы имеем на текущий момент**:
- Вопрос или же предикат, который мы используем для разбиения
- Функцию, бьющую исходную поданную выборку на 2 части согласно этому вопросу


**Что нам нужно**:
- Критерий разбиения (какой-нибудь один для примера)
- Information Gain
- Классы, методы для создания и работы с деревом

### Gini impurity

Более подробно см. https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
За основу реализации возьмем именно этот подход

In [81]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    
    for label in counts:
        label_probability = counts[label] / float(len(rows))
        impurity -= label_probability ** 2
        
    return impurity

Пара примеров для понимания того, как работает наш критерий.

Рассмотрим первый случай - полностью однородная выборка, тогда gini должен быть равен 1

In [82]:
non_mixed_data = ["Red", "Red"]
gini(non_mixed_data)

0.0

Рассмотрим второй случай - неоднородная выборка, в которой шанс неверной классификации 50%

In [83]:
mixed_data = ["Red", "Green"]
gini(mixed_data)

0.5

И третий - более неоднородная выборка, где шанс missclasfication еще выше

In [84]:
super_mixed_data = ["Red", "Green", "Orange"]
gini(super_mixed_data)

0.6666666666666665

### Information Gain

In [85]:
def info_gain(left, right, current, criterion=gini):
    """
    Information Gain.
    
    Значение критерия на текущей выборке минус взвешенное
    значение суммы левой и правой подвыборок.
    """
    
    p = float(len(left)) / (len(left) + len(right))
    current_uncertainty = criterion(current)
    
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

Посчитаем наш Information Gain после нашего сплита по предикату на красный цвет

In [71]:
question = Question(0, "Red")
question

Is color == Red

In [72]:
left, right = split_partition(training_data, Question(0, "Red"))

In [73]:
info_gain(left, right, training_data)

0.37333333333333324

In [74]:
left

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [75]:
right

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

Как видно выше, данный сплит является правильным с точки зрения прироста информации

Вернемся к тому, что у нас есть на текущий момент. Мы:

- Умеем разбивать исходную выборку на подвыборки
- У нас есть критерий того, насколько хороша текущая выборка с точки
  зрения однородности (Gini)
- У нас есть метрика, которая однозначно показывает, насколько хорошо
  наше разбиение на левые и правые подвыборки
  
Нам осталось:

- Для каждой выборки находить наилучшее разбиение
- Реализовать жадный алгоритм построения дерева

### Наилучшее разбиение выборки

Допустим исходная выборка состоит из нескольких записей. Каждая запись соответствует какому-то объекту,
в нашем случае это фрукт. У этого объекта есть несколько атрибутов - признаков.
Поэтому для того, чтобы найти наилучшее разбиение, надо пройтись по:

1) Признакам

2) Значениям этих признаков

Сравнивать различные разбиения между собой будем при помощи Information Gain,
который мы получили ранее. Наилучшее разбиение - то, которое имеет наибольший
Information Gain.

Имея данный четкий критерий, реализуем функцию *find_best_split*

In [89]:
def find_best_split(rows, debug=False):
    best_gain = 0
    best_question = None # предикат, дающий наибольший Information Gain
    current = rows
    n_features = len(rows[0]) - 1 # количество колонок с признаками
    
    # Сначала итерируемся по всем признакам, которые у нас есть
    for col_idx in range(n_features):
        values = set(row[col_idx] for row in rows)
        
        # Затем итерируемся по значениям этих признаков
        for val in values:
            split_question = Question(col_idx, val)
            
            left, right = split_partition(current, split_question)
            gain = info_gain(left, right, current, criterion=gini)
            
            if gain >= best_gain:
                best_gain = gain
                best_question = split_question
            
            if debug:
                print(f"Gain - {gain}, question - {split_question}")
                
    return best_gain, best_question

best_gain, best_question = find_best_split(training_data)

print(f"Best gain {best_gain} at question:\n{best_question}")

Best gain 0.37333333333333324 at question:
Is diameter >= 3


Теперь мы имеем функционал, который позволяет нам находить наилучший сплит.
На текущем этапе мы имеем все необходимое, чтобы реализовать решающее дерево.
Приступим к этому.

In [103]:
class Leaf:
    """
    Объект, который хранит в себе количество меток.
    Вся остальная информация будет храниться в узлах.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
class DecisionNode:
    """
    Узел нашего решающего дерева
    """
    def __init__(self, question, left_branch, right_branch):
        self.question = question
        self.left_branch = left_branch
        self.right_branch = right_branch
        
def build_tree(rows):
    """
    Рекурсивный алгоритм, условие остановки - Information Gain == 0
    """
    
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    left_rows, right_rows = split_partition(rows, question)
    left_branch = build_tree(left_rows)
    right_branch = build_tree(right_rows)
    
    return DecisionNode(question, left_branch, right_branch)

def print_tree(decision_tree, spacing=""):
    if isinstance(decision_tree, Leaf):
        print(spacing + "   predictions: " + str(decision_tree.predictions))
        return
    
    print(spacing + str(decision_tree.question))
    
    if decision_tree.right_branch:
        print(spacing + "--> True:")
        print_tree(decision_tree.right_branch, spacing + "   ")
    
    if decision_tree.left_branch:
        print(spacing + "--> False:")
        print_tree(decision_tree.left_branch, spacing + "   ")
    
tree = build_tree(training_data)

print_tree(tree)

Is diameter >= 3
--> True:
      predictions: {'Apple': 2}
--> False:
   Is diameter >= 2
   --> True:
         predictions: {'Lemon': 1}
   --> False:
         predictions: {'Grape': 2}


### Алгоритм обхода дерева

Теперь мы реализовали функционал построения и отрисовки дерева, осталось за малым - научиться предсказывать
метку класса поданного на вход объекта.

Сам алгоритм:
1. Берем входной объект + корень дерева в качестве рабочего (стартового) узла.

2. Берем вопрос (предикат), который соответствует этому узлу.

3. Если ответ на вопрос для данного входного объекта True, то идем по правой ветви, иначе - по левой.

4. Как только оказываемся не в узле, а в листе, обход завершаем, а в качестве предсказания берем тот класс, который
   встречается в листе чаще всего.

In [119]:
def make_predictions(node, row):
    """
    Сделаем рекурсивную реализацию
    """
    
    if isinstance(node, Leaf):
        return max(node.predictions, key=node.predictions.get)
    
    question_answer = node.question.match(row)
    
    if question_answer == True:
        return make_predictions(node.right_branch, row)
    else:
        return make_predictions(node.left_branch, row)
        
pred = make_predictions(tree, ['Yellow', 2])
print(f"Predicted label from tree - {pred}")

Predicted label from tree - Lemon


### Задание (опциональное)

1. Вместо Gini подставить энтропию Шеннона и посмотреть на результат
2. Поиграться с обучающей выборкой
3. Реализовать критерий остановы max_depth
3. Реализовать критерий остановы min_samples_leaf

## Часть 2. Практика

### Стратегии валидации

На последнем занятии были вопросы про валидацию, поэтому более подробно разберем эту тему

### train_test_split

Простой метод, который бьет вашу исходную выборку на 2 части - на train и test (hold-out множество).

Сгенерируем данные для начала

In [142]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [153]:
X = np.random.normal(size=(1000, 2))
y = [0 if i < 700 else 1 for i in range(1000)]

data = pd.DataFrame()
data["x0"] = X[:, 0]
data["x1"] = X[:, 1]
data["y"] = y

data.head()

,x0,x1,y
0,-0.359447,-1.865899,0
1,0.061335,0.950723,0
2,-0.105923,-0.082355,0
3,-0.915624,-0.183605,0
4,-0.301842,0.908814,0


Теперь, когда у нас есть данные, наша задача их разбить на 2 части - train выборку и test выборку.

Воспользуемся для этого методом train_test_split

In [154]:
feature_cols = [col for col in data.columns if col != 'y']

X_train, X_test, y_train, y_test = train_test_split(data[feature_cols], data.y, test_size=0.3, shuffle=False)

In [155]:
X_train.shape[0], X_test.shape[0]

(700, 300)

Обратим внимание на параметр shuffle = False.
Если shuffle на этапе разбиения выключен, а в test_size вы указали 0.3, это означает что:

* Размер train - 70% исходной выборки
* Размер тест - 30% исходной выборки

Помимо этого - в train попадет первые 70% записей, в test - 30% последних записей. Т.е. в каком порядке данные шли,
в таком порядке они и будут разбиты.
Давайте убедимся в этом, посмотрев на верхнюю часть X_train и сравнив с первым записями в data (data.head() выше).

In [156]:
X_train[:5]

,x0,x1
0,-0.359447,-1.865899
1,0.061335,0.950723
2,-0.105923,-0.082355
3,-0.915624,-0.183605
4,-0.301842,0.908814


Как видим - ровно это и произошло. Какие могут быть последствия?

Допустим следующее - у нас в датасете 70 процентов ноликов, 30 процентов единичек. Если они расположены так, что сначала идут нолики, а потом только 1, то в train попадут только нули, в test - единички. Поэтому обучать модель на таком разбиении не имеет смысла, т.к. она просто переобучится под классификацию нулей.

Посмотрим более внимательно на наши данные (мы их сгенерировали именно в таком порядке).

In [159]:
np.unique(y_train), np.unique(y_test)

(array([0]), array([1]))

Как видим, действительно в обучающей выборке нули, а в тестовой - единицы. Какой может быть выход из этой ситуации?

Попробуем включить shuffle на разбиении и посмотрим на результат.

In [174]:
X_train, X_test, y_train, y_test = train_test_split(data[feature_cols], data.y, test_size=0.3, shuffle=True)

In [175]:
np.unique(y_train), np.unique(y_test)

(array([0, 1]), array([0, 1]))

In [176]:
vals, train_counts = np.unique(y_train, return_counts=True)
print(f"Values - {vals}")
print(f"Counts - {train_counts}")
print(f"Frequencies - {train_counts[0] / X_train.shape[0]}, {train_counts[1] / X_train.shape[0]}")

Values - [0 1]
Counts - [491 209]
Frequencies - 0.7014285714285714, 0.2985714285714286


In [177]:
vals, test_counts = np.unique(y_test, return_counts=True)
print(f"Values - {vals}")
print(f"Counts - {test_counts}")
print(f"Frequencies - {test_counts[0] / X_test.shape[0]}, {test_counts[1] / X_test.shape[0]}")

Values - [0 1]
Counts - [209  91]
Frequencies - 0.6966666666666667, 0.30333333333333334


Теперь, вместо того, чтобы сразу разбить исходную выборку на train и test, sklearn
сначала рандомно перемешивает исходный датасет, и только потом делает разбиение.
Плюс данного подхода в том, что этот процесс случайный, поэтому балансы классов будут более-менее
одинаковы в train и test'е.

Теперь вспомним про решающие деревья и их свойство к переобучению. Учитывая данную особенность, нам необходимо, чтобы
в train и test были одинаковые пропорции 0 и 1. Если эти пропорции будут отличаться, то:

* Модель сильнее переобучится под один класс (и недообучится под другой)
* Значение метрик качества на тестовой выборке не будет отражать то, насколько хорошо дерево будет отделять один класс от другого.

Поэтому нам необходимо, чтобы соотношение между 0 и 1 было одинаковым как в test, так и в train (70% нолики, 30% единички). Сделать это можно,
воспользовавшись опцией stratify

In [178]:
X_train, X_test, y_train, y_test = train_test_split(data[feature_cols], data.y, test_size=0.3, stratify=data.y)

In [179]:
vals, train_counts = np.unique(y_train, return_counts=True)
print(f"Values - {vals}")
print(f"Counts - {train_counts}")
print(f"Frequencies - {train_counts[0] / X_train.shape[0]}, {train_counts[1] / X_train.shape[0]}")

Values - [0 1]
Counts - [490 210]
Frequencies - 0.7, 0.3


In [180]:
vals, test_counts = np.unique(y_test, return_counts=True)
print(f"Values - {vals}")
print(f"Counts - {test_counts}")
print(f"Frequencies - {test_counts[0] / X_test.shape[0]}, {test_counts[1] / X_test.shape[0]}")

Values - [0 1]
Counts - [210  90]
Frequencies - 0.7, 0.3


На данном примере видим, что именно это мы и получили

### KFold валидация

Пожалуй, самая распространенная схема кросс-валидации. Эмпирический метод оценки качества моделей.
Основная идея - делаем множества разбиений исходной выборки на train/test, на каждом разбиении
обучаем модель и замеряем качество, а затем усредняем это качество, получая итоговое значение.

<img src='img/kfolds.png' Width=800>

В чем достоинство данного подхода? Модель обучается и тестируется на разных подвыборках, поэтому
на основании разбоса качества можно судить о том, насколько сильно мы переобучаемся, ведь в случае хорошей
обобщающей способности он будет минимальным.

Пример использования ниже.

In [185]:
from sklearn.model_selection import KFold

folds = KFold(5, shuffle=False) # 5 - стандартное количество, которого обычно хватает для оценки

for i, (train_idx, val_idx) in enumerate(folds.split(data[feature_cols], data.y)):
    print(f"Fold #{i}")
    X_train = data[feature_cols].values[train_idx]
    y_train = data.y.values[train_idx]
    X_test = data[feature_cols].values[val_idx]
    y_test = data.y.values[val_idx]
    
    train_vals, train_counts = np.unique(y_train, return_counts=True)
    test_vals, test_counts = np.unique(y_test, return_counts=True)
    
    print(f"train values - {train_vals}, test_values - {test_vals}")

Fold #0
train values - [0 1], test_values - [0]
Fold #1
train values - [0 1], test_values - [0]
Fold #2
train values - [0 1], test_values - [0]
Fold #3
train values - [0 1], test_values - [0 1]
Fold #4
train values - [0 1], test_values - [1]


Если не делать shuffle, то аналогично примеру выше с train_test_split, может оказаться так, что
в train будут какие-то одни объекты, а в test - другие, поэтому оценка качества будет неправильной.
Чтобы этого избежать, необходимо использовать KFold c shuffle = True.

In [189]:
from sklearn.model_selection import KFold

folds = KFold(5, shuffle=True)

for i, (train_idx, val_idx) in enumerate(folds.split(data[feature_cols], data.y)):
    print(f"Fold #{i}")
    X_train = data[feature_cols].values[train_idx]
    y_train = data.y.values[train_idx]
    X_test = data[feature_cols].values[val_idx]
    y_test = data.y.values[val_idx]
    
    train_vals, train_counts = np.unique(y_train, return_counts=True)
    test_vals, test_counts = np.unique(y_test, return_counts=True)

    print(f"train freqs - {train_counts[0] / X_train.shape[0]}, {train_counts[1] / X_train.shape[0]}")
    print(f"test freqs - {test_counts[0] / X_test.shape[0]}, {test_counts[1] / X_test.shape[0]}")

Fold #0
train freqs - 0.69375, 0.30625
test freqs - 0.725, 0.275
Fold #1
train freqs - 0.70625, 0.29375
test freqs - 0.675, 0.325
Fold #2
train freqs - 0.6975, 0.3025
test freqs - 0.71, 0.29
Fold #3
train freqs - 0.6975, 0.3025
test freqs - 0.71, 0.29
Fold #4
train freqs - 0.705, 0.295
test freqs - 0.68, 0.32


Для того, чтобы соотношения классов были одинаковыми во всех fold'ах (разбиениях),
нужно использовать StratifiedKFold, указав колонку, по которой необходимо делать стратифицированное разбиение.

In [190]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(5)

for i, (train_idx, val_idx) in enumerate(folds.split(data[feature_cols], data.y, data.y)):
    print(f"Fold #{i}")
    X_train = data[feature_cols].values[train_idx]
    y_train = data.y.values[train_idx]
    X_test = data[feature_cols].values[val_idx]
    y_test = data.y.values[val_idx]
    
    train_vals, train_counts = np.unique(y_train, return_counts=True)
    test_vals, test_counts = np.unique(y_test, return_counts=True)

    print(f"train freqs - {train_counts[0] / X_train.shape[0]}, {train_counts[1] / X_train.shape[0]}")
    print(f"test freqs - {test_counts[0] / X_test.shape[0]}, {test_counts[1] / X_test.shape[0]}")

Fold #0
train freqs - 0.7, 0.3
test freqs - 0.7, 0.3
Fold #1
train freqs - 0.7, 0.3
test freqs - 0.7, 0.3
Fold #2
train freqs - 0.7, 0.3
test freqs - 0.7, 0.3
Fold #3
train freqs - 0.7, 0.3
test freqs - 0.7, 0.3
Fold #4
train freqs - 0.7, 0.3
test freqs - 0.7, 0.3


И ровно это и произошло, т.к. в метод split третьим аргументом мы передали колонку, по которой нужно подсчитать соотношение между 0 и 1, и именно в этих пропорциях делать разбиения.

### Задание

Самостоятельно реализовать на pandas аналог стратифицированный train_test_split.

Подсказки:

- метод sample из Pandas
- left outer join